# Imports

# USE TENSORBOARD PLEASE CHRIST I CAN"T TELL ALL THE TRIALS RESULTS

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

import pickle
import psycopg2

/home/johnrivera/.virtualenvs/ds_reddit_build-L6udXIR8/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Read in data

In [2]:
with open("../database/secrets", "r") as file:
    secrets = [i.strip('\n') for i in file.readlines()]


def conn_curs():
    """
    makes a connection to the database dont worry these are dummy keys
    """

    connection = psycopg2.connect(dbname=secrets[4], user=secrets[4],
                                  password=secrets[5], host=secrets[6])
    cursor = connection.cursor()
    return connection, cursor

In [3]:
conn, curs = conn_curs()
df = pd.read_sql("SELECT * FROM posts", conn)

# Inspect

In [4]:
df.drop('id', axis=1, inplace=True)

In [5]:
df.shape

(23951, 2)

In [6]:
df.head()

,text,subreddit
0,Thousand Year Blood War Arc Anime Adaptation M...,bleach
1,Burn The Witch - Chapter 4 Discussion Thread #...,bleach
2,Let the journey begin.,bleach
3,"Since Ichigos an English Literature major, thi...",bleach
4,I just made these Ulquiorra customs for a clie...,bleach


# Vectorize

In [7]:
le = LabelEncoder()
le.fit(df.subreddit.unique())

LabelEncoder()

In [8]:
le.classes_

array(['Animesuggest', 'AusLegal', 'BackYardChickens', 'BeardedDragons',
       'Bedbugs', 'Beekeeping', 'Berserk', 'BokuNoHeroAcademia',
       'CaptainTsubasaDT', 'CasualConversation', 'DDLC',
       'DecidingToBeBetter', 'Dragonballsuper', 'GetMotivated',
       'GiftIdeas', 'Gundam', 'Gunpla', 'INeedAName', 'IWantOut',
       'ImmigrationCanada', 'KissAnime', 'LegalAdviceUK', 'LifeProTips',
       'MLPLounge', 'NameThatSong', 'NoStupidQuestions', 'Petloss',
       'Pets', 'RBI', 'ShingekiNoKyojin', 'TooAfraidToAsk',
       'WouldYouRather', 'anime', 'answers', 'araragi', 'ask', 'asklaw',
       'ballpython', 'bettafish', 'bleach', 'cats', 'changemyview',
       'christmas', 'dbz', 'deathnote', 'declutter', 'dogs', 'ferrets',
       'findapath', 'getdisciplined', 'getting_over_it', 'guineapigs',
       'hamsters', 'help', 'ifyoulikeblank', 'immigration', 'legal',
       'legaladvice', 'legaladviceofftopic', 'leopardgeckos',
       'makemychoice', 'manga', 'needadvice', 'nosurf', 'pa

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.text, le.transform(df.subreddit), test_size=0.33, random_state=42)

In [10]:
vect = TfidfVectorizer(max_df=.95, min_df=80)

In [11]:
vect.fit(X_train)

TfidfVectorizer(max_df=0.95, min_df=80)

In [12]:
train_df = pd.DataFrame(vect.transform(X_train).todense(), columns=vect.get_feature_names())
train_df.head()

,000,10,100,11,12,13,14,15,16,17,...,yet,you,young,younger,your,youre,yourself,youtu,youtube,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.099524,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.067477,0.0,0.0,0.046306,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.087335,0.0,0.0,0.0,...,0.0,0.072734,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [13]:
val_df = pd.DataFrame(vect.transform(X_test).todense(), columns=vect.get_feature_names())
val_df.head()

,000,10,100,11,12,13,14,15,16,17,...,yet,you,young,younger,your,youre,yourself,youtu,youtube,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.041829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.053652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Model

In [14]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=train_df.shape[1]))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(df.subreddit.nunique(), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(train_df.to_numpy(), y_train, batch_size=32, epochs=30, validation_data=(val_df.to_numpy(), y_test))

Epoch 1/30
502/502 [==============================] - 1s 3ms/step - loss: 3.3819 - accuracy: 0.1991 - val_loss: 2.4659 - val_accuracy: 0.3430
Epoch 2/30
502/502 [==============================] - 1s 3ms/step - loss: 2.1842 - accuracy: 0.4110 - val_loss: 2.2741 - val_accuracy: 0.3925
Epoch 3/30
502/502 [==============================] - 1s 3ms/step - loss: 1.8766 - accuracy: 0.4758 - val_loss: 2.2305 - val_accuracy: 0.4068
Epoch 4/30
502/502 [==============================] - 1s 3ms/step - loss: 1.6852 - accuracy: 0.5232 - val_loss: 2.2767 - val_accuracy: 0.4088
Epoch 5/30
502/502 [==============================] - 1s 3ms/step - loss: 1.5211 - accuracy: 0.5659 - val_loss: 2.3063 - val_accuracy: 0.4181
Epoch 6/30
502/502 [==============================] - 1s 3ms/step - loss: 1.3736 - accuracy: 0.6035 - val_loss: 2.3688 - val_accuracy: 0.4130
Epoch 7/30
502/502 [==============================] - 1s 3ms/step - loss: 1.2252 - accuracy: 0.6456 - val_loss: 2.4987 - val_accuracy: 0.4093
Epoch 

### Early findings: A simple nueral net seems to be competitive with a simple logistic regression. As more the conventional models, out of the box it seems that the models worth considering are SVM, LogReg, SGD. SVM is slow on training and predictions though.

In [16]:
model = Sequential()
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.003), input_dim=train_df.shape[1]))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.003)))
model.add(Dropout(.2))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.003)))
model.add(Dropout(.2))
model.add(Dense(df.subreddit.nunique(), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_df.to_numpy(), y_train, batch_size=32, epochs=30, validation_data=(val_df.to_numpy(), y_test))

Epoch 1/30
502/502 [==============================] - 1s 2ms/step - loss: 4.4047 - accuracy: 0.0199 - val_loss: 4.1315 - val_accuracy: 0.0256
Epoch 2/30
502/502 [==============================] - 1s 2ms/step - loss: 4.0954 - accuracy: 0.0305 - val_loss: 4.0318 - val_accuracy: 0.0286
Epoch 3/30
502/502 [==============================] - 1s 2ms/step - loss: 4.0429 - accuracy: 0.0346 - val_loss: 4.0051 - val_accuracy: 0.0404
Epoch 4/30
502/502 [==============================] - 1s 2ms/step - loss: 4.0171 - accuracy: 0.0385 - val_loss: 3.9797 - val_accuracy: 0.0409
Epoch 5/30
502/502 [==============================] - 1s 2ms/step - loss: 3.9907 - accuracy: 0.0380 - val_loss: 3.9408 - val_accuracy: 0.0447
Epoch 6/30
502/502 [==============================] - 2s 3ms/step - loss: 3.9707 - accuracy: 0.0425 - val_loss: 3.9128 - val_accuracy: 0.0428
Epoch 7/30
502/502 [==============================] - 2s 3ms/step - loss: 3.9474 - accuracy: 0.0424 - val_loss: 3.9021 - val_accuracy: 0.0444
Epoch 

# IF YOU WANT TO USE NN PICKLE LABEL ENCODER, REMEMBER TO MAKE A PREDICT FUNCTION FOR WHATEVER MODEL YOU CHOOSE